In [1]:
import surfradpy.realtime_aod as sufrtaod

In [2]:
rtaod = sufrtaod.mfrsr_AOD_lev0()

In [3]:
rtaod.workplan

There are 92 files where the site in the file does not match the site of the folder...they will be skipped!!


,p2f,site,p2f_out
date,,,
2022-09-11,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2022/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...
2022-09-12,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2022/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...
2022-09-13,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2022/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...
2024-06-23,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2024/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...
2024-06-24,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2024/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...
...,...,...,...
2024-10-24,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2024/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...
2024-10-25,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2024/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...
2024-10-26,/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2024/t...,tbl,/nfs/grad/surfrad/products_level2/aod3realtime...


# process by hand, e.g. to find problems or do improvements

This is basically the guts of the "make_product" function

In [9]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as atmsrf


In [5]:
self = rtaod

In [4]:
row = rtaod.workplan.iloc[0]

In [6]:
row

p2f        /nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2022/t...
site                                                     tbl
p2f_out    /nfs/grad/surfrad/products_level2/aod3realtime...
Name: 2022-09-11 00:00:00, dtype: object

In [7]:
path2file = row.p2f
path2file

PosixPath('/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/tu/2022/tbl_mfrsr_20220911_D824_3126_660.tu')

In [11]:
langley_version = 0.4

In [13]:
lands = self.lands

In [14]:
# def make_product(self, path2file, product_version, verbose=False, langley_version='0.4', lands=None, aod_lim_max=2.8):
# path2file = '/nfs/grad/Inst/MFR/SURFRAD/tbl/mfrsr/ccc/2021/tbl20210301_0660.ccc'
p2f_ccc = pl.Path(path2file)
sii = atmsrf.read_ccc(p2f_ccc)
sii.direct_normal_irradiation.settings_calibration = 'atm_gam'
sii.direct_normal_irradiation.path2absorption_correction_ceoff_1625 = '/home/grad/htelg/data/grad/surfrad/aod1625/1625nm_absorption_correction_coefficience.nc'
sii.direct_normal_irradiation._apply_calibration_atm_gam(p2fld=f'/home/grad/htelg/data/grad/surfrad/mfrsr/langleys_concat.{langley_version}/tbl/',
                                                         th=0.02,
                                                         order_stderr=2,
                                                         lam_overtime=2.5e4,
                                                         ns_overtime=2,
                                                         lam_season=1e4,
                                                         ns_season=6,
                                                         lands=lands)
lands = sii.direct_normal_irradiation.lands

langleys.load.gampredict.done


/home/grad/htelg/prog_dev/atm-py/atmPy/radiation/solar.py:124: UserWarning: Discarding nonzero nanoseconds in conversion.
  date = date.to_pydatetime()


In [18]:
lands.v0prediction_gam.dataset.to_netcdf('/home/grad/htelg/data/grad/surfrad/mfrsr/langleys_gam_tbl.nc')

In [ ]:
ds = sii.direct_normal_irradiation.od_co2_ch4_h2o.transpose('datetime', 'channel')
ds = ds.rename_vars({var: f'od_{var.split("_")[0]}' for var in ds})
ds['aod'] = sii.direct_normal_irradiation.aod.transpose('datetime', 'channel')
ds['od_rayleigh'] = sii.direct_normal_irradiation.od_rayleigh.transpose('datetime', 'channel')
ds['od_total'] = sii.direct_normal_irradiation.od_total.transpose('datetime', 'channel')


#### QF exceeds detectionlimit aod max
dshighlim = (sii.direct_normal_irradiation.raw_data.global_horizontal_irradiation -
             sii.direct_normal_irradiation.raw_data.diffuse_horizontal_irradiation) < aod_lim_max
ds['flag_exceeds_aod_max'] = dshighlim
ds.flag_exceeds_aod_max.attrs[
    'info'] = f'The difference between global and diffuse is smaller than {aod_lim_max}, which is ~4 standard deviations of the measurement noise.'

# invalidate all date where certain flags are set
ds = ds.where(~ds.flag_exceeds_aod_max, np.nan)

#### angstrom exp
if 1:
    aod = atmcop.AOD_AOT(ds.aod)
    angcombos = [(415, 670), (500, 870), (500, 1625), (870, 1625)]
    anglist = []
    # assert(False), 'aod is not defined anywhere!?!?' #something like: aodinst = atmcop.AOD_AOT(aod)
    for angcomb in angcombos:
        # col1 = 415
        # col2 = 673
        ang = aod.aod2angstrom_exponent(
            column_1=angcomb[0], column_2=angcomb[1],)
        coordval = f'{angcomb[0]}_{angcomb[1]}'
        anglist.append(ang.expand_dims({'ang_channels': [coordval]}))
    ang = xr.concat(anglist, dim='ang_channels')
    ds['angstrom_exp'] = ang.transpose('datetime', 'ang_channels')

#### legacy cloudmask
ds['cloudmask_michalsky'] = aod.cloudmask.cloudmask_michalsky

#### temporary
ds['transmission'] = sii.direct_normal_irradiation.transmission
ds['global_horizontal'] = sii.direct_normal_irradiation.raw_data.global_horizontal_irradiation
ds['direct_normal'] = sii.direct_normal_irradiation.raw_data.direct_normal_irradiation
ds['diffuse_horizontal'] = sii.direct_normal_irradiation.raw_data.diffuse_horizontal_irradiation

sunpos = sii.direct_normal_irradiation.sun_position.copy()
sunpos = sunpos.drop(['ampm'], axis=1)
sunpos.columns.name = 'sun_params'
ds['sun_position'] = sunpos

# end temporary
ds.attrs = {att: sii.direct_normal_irradiation.raw_data.attrs[att]
            for att in sii.direct_normal_irradiation.raw_data.attrs if 'site' in att}
ds.attrs['MFRSR_serial_no'] = sii.direct_normal_irradiation.raw_data.attrs['serial_no']
ds.attrs['creation_timestamp'] = str(pd.Timestamp.now())
ds.attrs['product_version'] = product_version
# temporary variables for testing
# is this the Langley claibration part, we might still need this
if 0:
    V0df = sii.direct_normal_irradiation.tp_V0df
    V0df.index.name = 'channel'
    V0da = V0df.iloc[:, 0].to_xarray()
    V0da = V0da.expand_dims(
        {'date': [pd.to_datetime(pd.to_datetime(ds.datetime.values[0]).date())]})
#
    ds['V0'] = V0da
    ds['sun_earth_distance'] = sii.direct_normal_irradiation.tp_sedistcorr.iloc[:, 0]
    # .iloc[:,0]
    ds['sun_earth_V0_correction'] = sii.direct_normal_irradiation.tp_calib_interp_secorr
# output
out = {'product': ds}
out['sii'] = sii
out['aod'] = aod
if 0:
    out['VOdf'] = sii.direct_normal_irradiation.tp_V0df
return out